In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

# Start Spark Session
spark = SparkSession.builder \
    .master('local[4]') \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.cores", "4") \
    .config("spark.cores.max", "4") \
    .appName('pubg_data') \
    .getOrCreate()
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

# Readme 에 PUBG 그래픽 넣고 시작하는거 잊지말자

In [5]:
# 스파크 SQL 사용하여 원하는 df를 추출
# requirement:
# Cheater - 2020-03-01 이전 데이터만 사용해야함
    # Banned 당했다가 1년 후 풀려서 다시 플레이하는 유저들의 최근 플레이 게임을 배제해야함
#------여기까지 함------
# Cheater - 최소 banned 되기 전까지 4일간 데이터가 있어야함 만약에 없다면 없는 row는 평균값을 정해서 넣어주기
    # nickname당 started_at 이 같은 row는 average 함 (하루당 row 하나 되도록)
# 둘다 type 이 official 아닌 것 (event)빼야함
# 새로운 column "class"을 만들어서 Assign: cheater=1, normal=0

# 최근 4일간 x=started at, y=kills, groupby nickname 으로 histogram 겹치게 그리기.
# mode 에서 가장 많이 사용하는 tpp만 사용하는걸 justify 하는 설명 한줄정도 쓰고 tpp만 ㄱㄱ
    # fpp 와 tpp가 있는데 이거 count 비교하고 사실상 나머지는 이벤트라는것 이야기하기.


df = spark.sql("SELECT *\
               FROM parquet.`./cheater_data.parquet`\
               WHERE started_at < '2020-03-01'\
               ")
df = df.toPandas()
df

,type,mode,started_at,total_rank,queue_size,map_name,match_type,rank,heals,boosts,...,time_survived,kills,assists,kill_steaks,headshot_kills,longest_kill,damage_dealt,knock_downs,revives,nickname
0,official,tpp,2020-02-22 03:11:00,27,4.0,Savage_Main,None,27,0,0,...,1072,0,0,0,0,0,0,0,0,jjunbang
1,official,tpp,2020-02-22 02:49:52,29,4.0,Baltic_Main,None,2,10,5,...,1360,16,2,3,5,369,1624,12,1,jjunbang
2,official,tpp,2020-02-22 02:35:05,29,4.0,Savage_Main,None,1,3,6,...,1374,12,0,2,4,154,1933,14,0,jjunbang
3,official,tpp,2020-02-22 02:33:14,33,4.0,Baltic_Main,None,18,6,4,...,819,8,0,2,1,28,833,6,0,jjunbang
4,official,tpp,2020-02-22 02:28:59,29,4.0,Savage_Main,None,25,1,0,...,269,1,0,1,0,2,154,1,0,jjunbang
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230917,official,tpp,2020-02-08 15:50:41,28,4.0,Desert_Main,None,7,0,0,...,1529,1,0,1,0,90,136,1,0,8alcleaner7
230918,official,tpp,2020-02-08 15:36:37,21,4.0,Summerland_Main,None,6,1,0,...,285,0,1,0,0,0,0,0,0,8alcleaner7
230919,official,tpp,2020-02-16 15:45:06,28,4.0,Baltic_Main,None,2,0,0,...,243,0,0,0,0,0,81,0,0,BORAMSANGJO_ACE
230920,official,tpp,2020-02-16 05:13:27,27,4.0,Desert_Main,None,3,6,12,...,1656,1,0,1,0,22,427,3,1,a4zk-_-


In [7]:
df.nunique()

type                   2
mode                   8
started_at        175987
total_rank           100
queue_size             3
map_name               7
match_type             0
rank                 100
heals                 43
boosts                26
death_type             5
time_survived       2037
kills                 79
assists               22
kill_steaks           27
headshot_kills        53
longest_kill         701
damage_dealt        3298
knock_downs           40
revives                9
nickname            3271
dtype: int64

In [10]:
df['mode'].unique()

array(['tpp', 'warmodetpp', 'flareguntpp', 'flaregun', '20180324_rifle',
       'crossbowtpp', 'fpp', 'tequilatpp'], dtype=object)

In [ ]:
# df 만들어서 toPandas() 사용하여 visualization 시작
# 예) hi = df_cheater.describe().toPandas()

In [13]:
df_normal.describe().toPandas()

,summary,type,mode,total_rank,queue_size,map_name,match_type,rank,heals,boosts,...,time_survived,kills,assists,kill_steaks,headshot_kills,longest_kill,damage_dealt,knock_downs,revives,nickname
0,count,252461,252461,252461,241574,202800,62892,252461,252461,252461,...,252461,252461,252461,252461,252461,252461,252461,252461,252461,252461
1,mean,None,None,39.04639132380843,2.9403743780373715,None,None,19.28487172276114,1.7368623272505457,1.5492135418935995,...,751.533523989844,1.406712323883689,0.3905474508934053,0.5990905526002036,0.3528663833225726,33.882861115182145,197.09533749767292,0.8857486898966573,0.17752841032872405,2.290366605E8
2,stddev,None,None,23.0682153594538,1.2069927447550923,None,None,17.487201928451128,3.117925240872231,2.1814956748392236,...,534.0592511826246,2.6654090478058414,1.0206510668784392,0.7894534534051939,0.9077768744823802,71.71628125769156,306.9732686903104,1.5236061836788855,0.49019683213686255,3.9681698899965733E8
3,min,event,,1,1,Baltic_Main,arcade,0,0,0,...,-1935,0,0,0,0,0,0,0,0,010-010
4,max,official,warmodetpp,101,4,Summerland_Main,training,100,66,21,...,2204,59,23,15,33,8608,6830,30,15,zwaffer


<br>

In [5]:
# Import user data to PANDAS df
pd_df_cheater = pd.read_parquet("./cheater_data.parquet")
pd_df_normal = pd.read_parquet("./normal_user_data.parquet")

In [6]:
# Import user data to SPARK df
df_cheater = spark.read.parquet("./cheater_data.parquet")
df_normal = spark.read.parquet("./normal_user_data.parquet")

In [13]:
df_normal.describe().toPandas()

,summary,type,mode,total_rank,queue_size,map_name,match_type,rank,heals,boosts,...,time_survived,kills,assists,kill_steaks,headshot_kills,longest_kill,damage_dealt,knock_downs,revives,nickname
0,count,252461,252461,252461,241574,202800,62892,252461,252461,252461,...,252461,252461,252461,252461,252461,252461,252461,252461,252461,252461
1,mean,None,None,39.04639132380843,2.9403743780373715,None,None,19.28487172276114,1.7368623272505457,1.5492135418935995,...,751.533523989844,1.406712323883689,0.3905474508934053,0.5990905526002036,0.3528663833225726,33.882861115182145,197.09533749767292,0.8857486898966573,0.17752841032872405,2.290366605E8
2,stddev,None,None,23.0682153594538,1.2069927447550923,None,None,17.487201928451128,3.117925240872231,2.1814956748392236,...,534.0592511826246,2.6654090478058414,1.0206510668784392,0.7894534534051939,0.9077768744823802,71.71628125769156,306.9732686903104,1.5236061836788855,0.49019683213686255,3.9681698899965733E8
3,min,event,,1,1,Baltic_Main,arcade,0,0,0,...,-1935,0,0,0,0,0,0,0,0,010-010
4,max,official,warmodetpp,101,4,Summerland_Main,training,100,66,21,...,2204,59,23,15,33,8608,6830,30,15,zwaffer


In [25]:
pd_df_cheater.describe()

,total_rank,queue_size,rank,heals,boosts,time_survived,kills,assists,kill_steaks,headshot_kills,longest_kill,damage_dealt,knock_downs,revives
count,262312.000000,258295.000000,262312.000000,262312.000000,262312.000000,262312.000000,262312.000000,262312.000000,262312.000000,262312.000000,2.623120e+05,262312.000000,262312.000000,262312.000000
mean,35.810954,3.100165,17.233321,1.545484,2.002859,692.185241,2.085661,0.387051,0.790669,0.526282,1.529218e+02,272.719978,1.383051,0.164301
std,20.611053,1.187441,16.044643,2.799417,2.650707,477.504851,3.930178,1.016217,1.052799,1.531060,5.564097e+04,435.749348,2.393153,0.463500
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,26.000000,2.000000,6.000000,0.000000,0.000000,256.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,11.000000,0.000000,0.000000
50%,29.000000,4.000000,14.000000,0.000000,1.000000,605.000000,1.000000,0.000000,1.000000,0.000000,4.000000e+00,117.000000,0.000000,0.000000
75%,47.000000,4.000000,23.000000,2.000000,3.000000,1081.000000,3.000000,0.000000,1.000000,1.000000,5.800000e+01,337.000000,2.000000,0.000000
max,100.000000,4.000000,100.000000,56.000000,26.000000,2223.000000,98.000000,24.000000,45.000000,67.000000,2.849732e+07,9601.000000,40.000000,11.000000


In [26]:
pd_df_normal.describe()

,total_rank,queue_size,rank,heals,boosts,time_survived,kills,assists,kill_steaks,headshot_kills,longest_kill,damage_dealt,knock_downs,revives
count,252461.000000,241574.000000,252461.000000,252461.000000,252461.000000,252461.000000,252461.000000,252461.000000,252461.000000,252461.000000,252461.000000,252461.000000,252461.000000,252461.000000
mean,39.046391,2.940374,19.284872,1.736862,1.549214,751.533524,1.406712,0.390547,0.599091,0.352866,33.882861,197.095337,0.885749,0.177528
std,23.068215,1.206993,17.487202,3.117925,2.181496,534.059251,2.665409,1.020651,0.789453,0.907777,71.716281,306.973269,1.523606,0.490197
min,1.000000,1.000000,0.000000,0.000000,0.000000,-1935.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,26.000000,2.000000,7.000000,0.000000,0.000000,263.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.000000,0.000000,0.000000
50%,30.000000,4.000000,15.000000,0.000000,1.000000,643.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000
75%,48.000000,4.000000,26.000000,2.000000,2.000000,1177.000000,2.000000,0.000000,1.000000,0.000000,39.000000,251.000000,1.000000,0.000000
max,101.000000,4.000000,100.000000,66.000000,21.000000,2204.000000,59.000000,23.000000,15.000000,33.000000,8608.000000,6830.000000,30.000000,15.000000


In [24]:
df_cheater.describe().toPandas()

,summary,type,mode,total_rank,queue_size,map_name,match_type,rank,heals,boosts,...,time_survived,kills,assists,kill_steaks,headshot_kills,longest_kill,damage_dealt,knock_downs,revives,nickname
0,count,262312,262312,262312,258295,252525,28529,262312,262312,262312,...,262312,262312,262312,262312,262312,262312,262312,262312,262312,262312
1,mean,None,None,35.81095413095856,3.100164540544726,None,None,17.233321388270458,1.545484003781756,2.0028591905822073,...,692.1852412394401,2.085661349842935,0.387050535240477,0.7906691268413187,0.5262816798316509,152.92179541919546,272.7199784988868,1.3830514806794962,0.16430052761596878,5.2001734373229E9
2,stddev,None,None,20.611052840070275,1.18744073446667,None,None,16.044642886756098,2.7994165874722583,2.6507068715989774,...,477.5048507227209,3.9301784962437254,1.0162165363765092,1.0527987671254624,1.5310598350492668,55640.97450660263,435.749347631955,2.3931527779094197,0.4635004668939277,5.463764400142923E10
3,min,event,,1,1,Baltic_Main,arcade,1,0,0,...,0,0,0,0,0,0,0,0,0,0-TTukBaeGi-0
4,max,official,warmodetpp,100,4,Summerland_Main,training,100,56,26,...,2223,98,24,45,67,28497320,9601,40,11,zzzz18year


In [23]:
df_normal.describe().show(vertical=True)

-RECORD 0------------------------------
 summary        | count                
 type           | 252461               
 mode           | 252461               
 total_rank     | 252461               
 queue_size     | 241574               
 map_name       | 202800               
 match_type     | 62892                
 rank           | 252461               
 heals          | 252461               
 boosts         | 252461               
 death_type     | 252461               
 time_survived  | 252461               
 kills          | 252461               
 assists        | 252461               
 kill_steaks    | 252461               
 headshot_kills | 252461               
 longest_kill   | 252461               
 damage_dealt   | 252461               
 knock_downs    | 252461               
 revives        | 252461               
 nickname       | 252461               
-RECORD 1------------------------------
 summary        | mean                 
 type           | null                 
